REQUIREMENTS: Besides the datasets and python libraries, you need to install the GT America fonts on your computer to see the completed graphs

QUESTION 1: Total BPD Budget vs. Previous Years

FY15 Adopted: $319700081
FY15 Approp: $319700081
FY15 Exp: $337310895

FY16 Adopted: $323509388
FY16 Approp: $332544971
FY16 Exp: $348887844

FY17 Adopted: $356341193
FY17 Approp: $364321048
FY17 Exp: $364594820

FY18 Adopted: $373814105
FY18 Approp: $385562872
FY18 Exp: $399924493

FY19 Adopted: $400425675
FY19 Approp: $400435574
FY19 Exp: $416762373

FY20 Adopted: $414237376
FY20 Approp: $414306876
FY20 Exp: $425553210

FY21 Adopted: $404182025
FY21 Approp: $404182025
FY21 Exp: $422917499

FY22 Adopted: $399871218
FY22 Approp: $400451303
 
FY23 Adopted: $395094796

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly_express as px

file_adopted_operating_budget_fy23 = pd.read_csv('./Data/operating-budget.csv')

budget_df_uncut = pd.DataFrame(file_adopted_operating_budget_fy23)

budget_df_withNA = budget_df_uncut[['Cabinet', 'Dept', 'Program   ', 'Expense Category',
       'FY20 Actual Expense', 'FY21 Actual Expense', 'FY22 Appropriation',
       'FY23 Adopted']]

budget_df = budget_df_withNA.dropna()

# Replace '#Missing' values with '0' so we can type cast the columns to numbers
budget_df = budget_df.replace('#Missing', '0')

# Convert datatype to strings 
budget_df['FY20 Actual Expense'] = budget_df['FY20 Actual Expense'].apply(lambda x: float(x.replace(",","").replace("$","")) if x != 'TOTAL_GROSS' and str(type(x)) == '<class \'str\'>' else x)
budget_df['FY21 Actual Expense'] = budget_df['FY21 Actual Expense'].apply(lambda x: float(x.replace(",","").replace("$","")) if x != 'TOTAL_GROSS' and str(type(x)) == '<class \'str\'>' else x)
budget_df['FY22 Appropriation'] = budget_df['FY22 Appropriation'].apply(lambda x: float(x.replace(",","").replace("$","")) if x != 'TOTAL_GROSS' and str(type(x)) == '<class \'str\'>' else x)
budget_df['FY23 Adopted'] = budget_df['FY23 Adopted'].apply(lambda x: float(x.replace(",","").replace("$","")) if x != 'TOTAL_GROSS' and str(type(x)) == '<class \'str\'>' else x)

# Get only data from Police Department
bpd_budget_df = budget_df.loc[budget_df['Dept'] == "Police Department"]

# Compute sums and add to list
bpd_budget = []
# bpd_budget.append(bpd_budget_df['FY20 Actual Expense'].sum())
# bpd_budget.append(bpd_budget_df['FY21 Actual Expense'].sum())
# bpd_budget.append(bpd_budget_df['FY22 Appropriation'].sum())  # Appropriated budget = means ?? (GUESS: Proposed budget (Needs approval until adopted!))
# bpd_budget.append(bpd_budget_df['FY23 Adopted'].sum())  # Adopted budget = 1 year spending fianncial plan
bpd_budget = [319700081, 323509388, 356341193, 373814105, 400425675, 414237376, 404182025, 399871218, 395094796]

fig1_dict = {
    'year': ['FY15', 'FY16', 'FY17', 'FY18', 'FY19', 'FY20', 'FY21', 'FY22', 'FY23'],
    'Budget': bpd_budget
}

pd.options.plotting.backend = "plotly"
df1 = pd.DataFrame(fig1_dict)
print(df1)
# using Plotly Express via the Pandas backend
fig1 = df1.plot.bar(x='year', y='Budget', color_discrete_sequence=['#EF404E'])
fig1.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray')

# Aesthetic customizations
fig1.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    font_family="GT America",
    font_color="black",
    xaxis_title=None,
    yaxis_title=None,
    yaxis_tickprefix = '$',
    margin=dict(
        l=100,
        r=100,
        b=80,
        t=90,
        pad=4
    ),
    height=475,
    width=1050
)

# Customize title to add captions!
fig1.update_layout(
    title=go.layout.Title(
        text="BPD's adopted budget has only slightly decreased in recent years after a long period of increases ",
        xref="paper",
        x=0,
        y=0.9,
        font_family="GT America Bold",
        font_size=18
    )
)

fig1.show()


   year     Budget
0  FY15  319700081
1  FY16  323509388
2  FY17  356341193
3  FY18  373814105
4  FY19  400425675
5  FY20  414237376
6  FY21  404182025
7  FY22  399871218
8  FY23  395094796


Note: FY 20 & 21 are Expenses, FY 22 and 23 are Budgets 
Police dept budget slightly decreased in recent years.

Improvement:
- We could change this to only budgets


QUESTION 2: BPD Budget vs. other departments budgets + change over time

Issues:
- Which departments do we include (excluding BPD) ?
- I look at Top 4 departments with highest budgets each year + BPD

In [3]:
num_depts = 5
year = ['FY20 Expense'] * num_depts + ['FY21 Expense'] * num_depts + ['FY22 Appropriated Budget'] * num_depts + ['FY23 Adopted Budget'] * num_depts

budget = []
dept = []

# Sum all budgets
summed_budget_df = budget_df.groupby(by=['Dept'], sort=False).sum()

# Will set the plot values based on all departments
def getTopOverallDepts():
    # Sort departments by budget sums
    fy20_sorted_df = summed_budget_df.sort_values(by=['FY20 Actual Expense'], ascending=False)
    fy21_sorted_df = summed_budget_df.sort_values(by=['FY21 Actual Expense'], ascending=False)
    fy22_sorted_df = summed_budget_df.sort_values(by=['FY22 Appropriation'], ascending=False)
    fy23_sorted_df = summed_budget_df.sort_values(by=['FY23 Adopted'], ascending=False)

    # Get top departments
    top_fy20_df = fy20_sorted_df.head(num_depts)
    top_fy21_df = fy21_sorted_df.head(num_depts)    
    top_fy22_df = fy22_sorted_df.head(num_depts)    
    top_fy23_df = fy23_sorted_df.head(num_depts)    

    for index, row in top_fy20_df.iterrows():
        dept.append(index)
        budget.append(row['FY20 Actual Expense'])

    for index, row in top_fy21_df.iterrows():
        dept.append(index)
        budget.append(row['FY21 Actual Expense'])

    for index, row in top_fy22_df.iterrows():
        dept.append(index)
        budget.append(row['FY22 Appropriation'])

    for index, row in top_fy23_df.iterrows():
        dept.append(index)
        budget.append(row['FY23 Adopted'])

getTopOverallDepts()

fig2_dict = {
    'year': year,
    'Budget': budget,
    'Department': dept
}

df2 = pd.DataFrame(fig2_dict)
color_discrete_sequence=['#d9dcf5','#EF404E','#a39fd7','#7774a6', "#494983", "#130F54"]
fig2 = px.bar(df2, x='year', y='Budget', color='Department', barmode="group", color_discrete_sequence=color_discrete_sequence)
fig2.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray')

# Aesthetic customizations
fig2.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    font_family="GT America",
    font_color="black",
    xaxis_title=None,
    yaxis_title=None,
    yaxis_tickprefix = '$',
    margin=dict(
        l=100,
        r=100,
        b=80,
        t=120,
        pad=4
    ),
    width=1050
)

# Customize title to add captions!
fig2.update_layout(
    title=go.layout.Title(
        text="The Boston Police Department budget consistently ranks as the second-highest, <br>surpassed only by Boston Public Schools.",
        xref="paper",
        x=0,
        font_family="GT America Bold",
        font_size=19
    )
)
fig2.show()



/var/folders/15/ngd639_x4tn_n4xh55wvc5lr0000gn/T/ipykernel_14666/3390850744.py:8: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



Analysis: 
BPD remained the department with the second highest budget each year, with small decrements over the years. 
BPS, Pensions, and Charter School Tuition all had budget increases over the years.

NOTE: Might be more useful to show percentage of total city budget that BPD takes up per year + change over time

In [4]:

num_depts = 5
year = ['FY20 Expense'] * num_depts + ['FY21 Expense'] * num_depts + ['FY22 Appropriated Budget'] * num_depts + ['FY23 Adopted Budget'] * num_depts
budget = []
dept = []

# REMOVE BPS
budget_wo_bps = budget_df[budget_df.Dept != 'Boston Public Schools']

# Sum all budgets
summed_budget_df = budget_wo_bps.groupby(by=['Dept'], sort=False).sum()

getTopOverallDepts()

fig2_2_dict = {
    'year': year,
    'Budget': budget,
    'Department': dept
}

print(dept)

df2_2 = pd.DataFrame(fig2_2_dict)
color_discrete_sequence=['#EF404E', '#d9dcf5', '#a39fd7', '#7774a6', "#494983", "#130F54"]
# color_discrete_sequence=['#130f54','#EF404E','#ecb6fa','#CCCCCC', "#fae1ab", "#b4fae9"]
fig2_2 = px.bar(df2_2, x='year', y='Budget', color='Department', barmode="group", color_discrete_sequence=color_discrete_sequence)
fig2_2.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray')

# Aesthetic customizations
fig2_2.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    font_family="GT America",
    font_color="black",
    xaxis_title=None,
    yaxis_title=None,
    yaxis_tickprefix = '$',
    margin=dict(
        l=100,
        r=100,
        b=80,
        t=120,
        pad=4
    ),
    width=1050
)

# Customize title to add captions!
fig2_2.update_layout(
    title=go.layout.Title(
        text="The Boston Police Department budget consistently ranks as the second-highest, <br>surpassed only by Boston Public Schools.",
        xref="paper",
        x=0,
        font_family="GT America Bold",
        font_size=18
    )
)

fig2_2.show()


['Police Department', 'Pensions', 'Fire Department', 'Health Insurance', 'Charter School Tuition', 'Police Department', 'Pensions', 'Fire Department', 'Charter School Tuition', 'Health Insurance', 'Police Department', 'Pensions', 'Fire Department', 'Charter School Tuition', 'Health Insurance', 'Police Department', 'Pensions', 'Fire Department', 'Charter School Tuition', 'Debt Service']


/var/folders/15/ngd639_x4tn_n4xh55wvc5lr0000gn/T/ipykernel_14666/2411399467.py:10: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



QUESTION 3: Number of police officers per 10,000 residents

Presentation: 
- Look at FBI uniform crime reporting average of police officers per 10,000 residents & compare it to boston
- FBI data sets here: https://crime-data-explorer.fr.cloud.gov/#
- Bar chart where each x-axis represent year and each year there will be 2 bars, 1 for average in nation, 1 for value in boston  
OR
- Scatter plot, each point is a city
- X axis is population
- Y axis is officers / 10,000 residents

NOTE: Some cities with low populations have 0 officers! (How do we handle this if at all?) 

In [5]:
import plotly.figure_factory as ff

fbi_law_enforcement_employee_data = pd.read_csv('./Data/FBI_law_enforcement_employee_data.csv')
employee_nums_1960_2021 = pd.DataFrame(fbi_law_enforcement_employee_data)
employee_nums_1960_2021.dropna()

# Get only city data
employee_nums_1960_2021 = employee_nums_1960_2021.loc[employee_nums_1960_2021['agency_type_name'] == 'City']

# Get only data from 2016-2021
employee_nums_2015_2021 = employee_nums_1960_2021.loc[employee_nums_1960_2021['data_year'].isin([2015, 2016, 2017, 2018, 2019, 2020, 2021])]



# Get Boston df
boston_df = employee_nums_2015_2021.loc[employee_nums_2015_2021["pub_agency_name"] == "Boston"]
boston_df = boston_df.loc[employee_nums_2015_2021["state_abbr"] == "MA"]
# boston_df = boston_df.sort_values(by='data_year', ascending=True)

def getValsFrom(dataset):

    # Declare variables to track statistics
    boston_vals = []
    average_vals = []
    median_vals = []
    city_vals_each_year = [] # List of lists where the first list is the values of all cities in 2015
    boston_percentiles = []
    cities_by_year = {} # Dictionary where key = year & value = list of ("city agency", offc/10k_res)

    for i in range(2015,2022):

        # Get Boston values
        row = boston_df.loc[boston_df["data_year"] == i]
        row = row.squeeze()
        pop = row["population"]
        officer_ct = row["officer_ct"]
        # pop * factor = 10,000 so factor = 10000/pop
        factor = 10000/pop
        bos_offc_per_10k = officer_ct * factor
        boston_vals.append(bos_offc_per_10k)

        # Reset these variables as they are tracked each year
        cities_measured = 0
        cities = []

        # Get average values
        i_year_rows = dataset.loc[dataset["data_year"] == i]
        all_cities_vals = []
        for index, row in i_year_rows.iterrows():
            pop = row["population"]
            if (pop == 0):
                break
            officer_ct = row["officer_ct"] 
            factor = 10000/pop
            offc_per_10k = officer_ct * factor
            # if (officer_ct == 0):
            #     print(str(row["data_year"]) + row['pub_agency_name'])
            # Ignore nan values
            if np.isnan(offc_per_10k):
                break
            cities_measured += 1
            all_cities_vals.append(offc_per_10k)
            cities.append((row["pub_agency_name"],offc_per_10k))

        # Compute statistices
        sum = np.sum(all_cities_vals)
        avg = sum/len(all_cities_vals)
        median = np.median(all_cities_vals)

        city_vals_each_year.append(all_cities_vals)
        median_vals.append(median)
        average_vals.append(avg)

        # Sort the all cities by offc/capita
        cities.sort(key = lambda x: x[1])
        # Add list to greater collection
        cities_by_year[i] = cities

        cities_w_less_offc_per_capita = np.where(all_cities_vals < bos_offc_per_10k)[0]

        percentile = round(((len(cities_w_less_offc_per_capita)/cities_measured) * 100), 2)

        # print("Total cities measured:" + str(cities_measured))
        # print("Total cities with less officers/capita than Boston: " + str(len(cities_w_less_offc_per_capita)))
        # print("Boston has more officers/capita than: " + str(percentile) + " percent of other city PDs")
        boston_percentiles.append(percentile)
    
    return {'boston_vals': boston_vals, 'average_vals': average_vals, 'median_vals': median_vals, 'city_vals_each_year': city_vals_each_year, 'boston_percentiles': boston_percentiles, 'cities_by_year': cities_by_year}

stat_vals = getValsFrom(employee_nums_2015_2021)

boston_vals = stat_vals['boston_vals']
average_vals = stat_vals['average_vals']
median_vals = stat_vals['median_vals']
city_vals_each_year = stat_vals['city_vals_each_year']
boston_percentiles = stat_vals['boston_percentiles']

print(boston_vals)
print(average_vals)
print(median_vals)

# Create plots
year = ["2015"] * 2 + ["2016"] * 2 + ["2017"] * 2 + ["2018"] * 2 + ["2019"] * 2 + ["2020"] * 2 + ["2021"] * 2
values = []
labels = []

for i in range(int(len(year)/2)):
    values.append(median_vals[i])
    values.append(boston_vals[i])
    labels.append("Median")
    labels.append("Boston")

# BAR CHART
fig3_dict = {
    'Year': year,
    'Officers per 10k residents': values,
    'Key': labels
}

df3 = pd.DataFrame(fig3_dict)
fig3 = px.bar(df3, 
    x='Year', 
    y='Officers per 10k residents', 
    color='Key', 
    barmode="group", 
    color_discrete_map={
        'Boston': '#EF404E',
        'Median': '#231F20'
    })

fig3.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    font_family="GT America",
    font_color="black",
    xaxis_title=None,
    margin=dict(
        l=75,
        r=75,
        b=100,
        t=100,
        pad=4
    ),
    width=1050
)

# Customize title to add captions!
fig3.update_layout(
    title=go.layout.Title(
        text="Boston consistently retains more officers per resident than the median of other cities",
        xref="paper",
        x=0,
        font_family="GT America Bold",
        font_size=20
    )
)

fig3.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig3.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray')
fig3.show()


# DISTRIBUTION GRAPHS
hist_data = [city_vals_each_year[-1]]
group_labels = ['Density Curve'] 
bos = boston_vals[-1] # Get Boston's 2021 value


colors = ['#0055AA']
fig4 = ff.create_distplot(hist_data, group_labels, show_rug=False, colors=colors)
fig4.update_layout(xaxis_title="Officers per 10,000 residents", yaxis_title="Frequency")
# Customize title to add captions!
fig4.update_layout(
    title=go.layout.Title(
        text="2021 Distribution of Officers Per 10,000 Residents",
        xref="paper",
        x=0,
        font_family="GT America Bold",
        font_size=20
    )
)
fig4.add_vline(x=bos, line_width=2, line_dash="dash", line_color="red", annotation_text="Boston officers per pop > " + str(boston_percentiles[-1]) + "% of cities")
fig4.update_layout(plot_bgcolor='rgba(0,0,0,0)', font_family="GT America", font_color="black", width=1050)
fig4.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray')
fig4.show()



[32.152939160446024, 31.53380423814329, 32.288626642436775, 30.546746454806794, 30.66067092930591, 30.57406682412598, 30.989360886999506]
[27.560430864150657, 30.48646375252761, 28.09017640608657, 28.993403126507168, 29.803901416835096, 32.702720379152915, 34.18413647794076]
[19.045823584714604, 20.161290322580648, 20.065882982459076, 24.975024975024972, 24.37403057832927, 27.286257648420705, 27.68167415700203]


FIGURE 3 Analysis:
- Boston constantly ranks higher than the median in terms of Officers per 10k residents
- Boston's officers/10k pop, has decreased over the years, while the median has increased

    ISSUES:
    - UCR (Uniform Crime Reports) are not mandatory! This explains the variation in the # of cities which submitted UCR data each year
    - Interestingly, the # of cities which submitted UCR datas has been steadily decreasing
    - Upwards trend of median could potentially be explained by this
    - POSSIBLE EXPLANATION: Larger cities (which typically have larger police forces) may continue to report UCR while smaller cities may have opted out

FIGURE 4 Analysis:
- Different way of displaying the "Officers / 10k residents" statistic
- We are able to see where Boston lies compared to other cities 
- I also display a percentage

    ISSUES:
    - I only plot values for 1 year (2021)
    - Can fix this by using: Box plot OR Ridgeline plot

In [6]:
# SAME THING BUT LOOKING @ CITIES ~ SAME SIZE AS BOSTON
# population_group_desc = 'Cities from 500,000 thru 999,999'
# Since 1995 - Present, Boston population has remained between 500k - 999k

employee_nums_500k_to_999k_pop = employee_nums_2015_2021.loc[employee_nums_2015_2021["population_group_desc"] == 'Cities from 500,000 thru 999,999']

print(len(employee_nums_500k_to_999k_pop))

stat_vals = getValsFrom(employee_nums_500k_to_999k_pop)

boston_vals = stat_vals['boston_vals']
average_vals = stat_vals['average_vals']
median_vals = stat_vals['median_vals']
city_vals_each_year = stat_vals['city_vals_each_year']
boston_percentiles = stat_vals['boston_percentiles'] # 0th index is earliest year, last index is latest (2021) year
cities_by_year = stat_vals['cities_by_year']

print(boston_vals)
print(average_vals)
print(median_vals)

# ----- BAR GRAPH (BOSTON VS MEDIAN) -----

# Create plots
year = ["2015"] * 2 + ["2016"] * 2 + ["2017"] * 2 + ["2018"] * 2 + ["2019"] * 2 + ["2020"] * 2 + ["2021"] * 2
values = []
labels = []

for i in range(int(len(year)/2)):
    values.append(median_vals[i])
    values.append(boston_vals[i])
    labels.append("Median")
    labels.append("Boston")

fig3_dict = {
    'Year': year,
    'Officers per 10k residents': values,
    'Key': labels
}

df3 = pd.DataFrame(fig3_dict)
fig3 = px.bar(df3, x='Year', y='Officers per 10k residents', color='Key', title='Officers Per 10,000 Residents (Boston vs cities with populations between 500,000 and 999,999)', barmode="group", color_discrete_map={
        'Boston': '#EF404E',
        'Median': '#231F20'
    })

fig3.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    font_family="GT America",
    font_color="black",
    xaxis_title=None,
    margin=dict(
        t=100,
        b=120,
        pad=4
    ),
    width=1050
)

# Subtitle
fig3.update_layout(
    title=go.layout.Title(
        text="Boston consistently retains more officers per resident than the median of similar sized cities <br><sup style='color:gray;'>Compared cities have populations between 500,000 and 999,999</sup>",
        xref="paper",
        x=0,
        font_family="GT America Bold",
        font_size=20
    )
)

fig3.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray')

# Create caption text
cities_2021 = cities_by_year[2021]
cities_and_offc_per_capita = list(map(list, zip(*cities_2021))) # Index 0 = cities list, index 1 = offc/resident list
cities_and_offc_per_capita[0].reverse()
cities_and_offc_per_capita[0][9] = 'Nashville'
caption_list = cities_and_offc_per_capita[0][:10] 
caption_txt = ', '.join(caption_list)
caption_txt = caption_txt[:69] + '<br>' + caption_txt[69:]

# Caption
fig3.add_annotation(xref='paper',
    yref='paper',
    x=0, y=-0.4,
    showarrow=False,
    text ="A total of 24 cities were compared in 2021.<br>Top 10 cities by officers/resident in 2021 (decending order): " + caption_txt,
    font_size=13,
    font_color="gray",
    font_family="GT America",
    align="left")

    

fig3.show()

# ----- BAR GRAPH (BOSTON VS AVG) -----

# Create plots
year = ["2015"] * 2 + ["2016"] * 2 + ["2017"] * 2 + ["2018"] * 2 + ["2019"] * 2 + ["2020"] * 2 + ["2021"] * 2
values = []
labels = []

for i in range(int(len(year)/2)):
    values.append(average_vals[i])
    values.append(boston_vals[i])
    labels.append("Average")
    labels.append("Boston")

fig3_dict = {
    'Year': year,
    'Officers per 10k residents': values,
    'Key': labels
}

df3 = pd.DataFrame(fig3_dict)
fig3 = px.bar(df3, x='Year', y='Officers per 10k residents', color='Key', title='Officers Per 10,000 Residents (Median of departments in cities w/ 500k - 999k population)', barmode="group", color_discrete_map={
        'Boston': '#EF404E',
        'Average': '#231F20'
    })
    
# Font
fig3.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    font_family="GT America",
    font_color="black",
    xaxis_title=None,
    margin=dict(
        t=100,
        b=120,
        pad=4
    ),
    width=1050
)

# Subtitle
fig3.update_layout(
    title=go.layout.Title(
        text="Boston consistently retains more officers per resident than the average of similar sized cities <br><sup style='color:gray;'>Compared cities have populations between 500,000 and 999,999</sup>",
        xref="paper",
        x=0,
        font_family="GT America Bold",
        font_size=20
    )
)

fig3.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray')

# Caption
fig3.add_annotation(xref='paper',
    yref='paper',
    x=0, y=-0.4,
    showarrow=False,
    text ="A total of 24 cities were compared in 2021.<br>Top 10 cities by officers/resident in 2021 (decending order): " + caption_txt,
    font_size=13,
    font_color="gray",
    font_family="GT America",
    align="left")

fig3.show()

# ----- DISTRIBUTION GRAPHS (Looks horrible so decided not to use) -----

# ----- BAR GRAPH (BY CITY) -----

cities_2021 = cities_by_year[2021]
cities_and_offc_per_capita = list(map(list, zip(*cities_2021))) # Index 0 = cities list, index 1 = offc/resident list
cities_and_offc_per_capita[0][5] = 'Louisville'
cities_and_offc_per_capita[0][11] = 'Charlotte'
cities_and_offc_per_capita[0][14] = 'Nashville'

fig5_dict = {
    'City or Agency Name': cities_and_offc_per_capita[0],
    'Officers per 10k residents': cities_and_offc_per_capita[1]
}

pd.options.plotting.backend = "plotly"
df5 = pd.DataFrame(fig5_dict)
# using Plotly Express via the Pandas backend

color_seq = ['#BBBBBB'] * 18 + ['#EF404E'] + ['#BBBBBB'] * 5

fig5 = px.bar(
    df5,
    x='City or Agency Name', 
    y='Officers per 10k residents', 
    color=color_seq,
    color_discrete_map="identity",
    category_orders={'City or Agency Name': cities_and_offc_per_capita[0]}
)

# Customize title to add captions!
fig5.update_layout(
    title=go.layout.Title(
        text="Boston has more officers per resident than 70% of other similar sized cities (2021) <br><sup style='color:gray;'>Compared cities have populations between 500,000 and 999,999</sup>",
        xref="paper",
        x=0,
        font_family="GT America Bold",
        font_size=20
    )
)

# Aesthetic customizations
fig5.update_layout(
    plot_bgcolor='rgba(0,0,0,0)', 
    height=525,
    font_family="GT America",
    font_color="black",
    xaxis_title=None,
    margin=dict(
        l=75,
        r=75,
        b=100,
        t=100,
        pad=4
    ),
    width=1050
)

fig5.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray')
fig5.show()

print(df5)


167
[32.152939160446024, 31.53380423814329, 32.288626642436775, 30.546746454806794, 30.66067092930591, 30.57406682412598, 30.989360886999506]
[24.00087693271406, 24.18327725129472, 23.700978027322375, 23.016098631372344, 23.24475517810718, 23.357219553702908, 22.868856261430654]
[19.963834937611765, 20.510821919459364, 20.073055942553353, 19.511723678770664, 19.079900232999, 20.624610290793925, 18.840629726588446]


   City or Agency Name  Officers per 10k residents
0             Portland                   11.906375
1           Sacramento                   13.832087
2               Fresno                   14.410653
3               Tucson                   14.740328
4              Seattle                   14.954002
5           Louisville                   15.304407
6                 Mesa                   16.468339
7        Oklahoma City                   16.471100
8              El Paso                   16.619520
9          Albuquerque                   16.662324
10          Fort Worth                   17.207146
11           Charlotte                   18.132727
12              Denver                   19.548533
13            Honolulu                   19.712974
14           Nashville                   20.997664
15       San Francisco                   24.601539
16         Kansas City                   25.670456
17           Milwaukee                   27.876232
18              Boston         

Question 4: 
BPD proposed vs adopted budget

Proposed Budget --> Adopted Budget --> Appropriated Budget --> Actual Expenditure

FY15 Adopted: $319700081
FY15 Approp: $319700081
FY15 Exp: $337310895

FY16 Adopted: $323509388
FY16 Approp: $332544971
FY16 Exp: $348887844

FY17 Adopted: $356341193
FY17 Approp: $364321048
FY17 Exp: $364594820

FY18 Adopted: $373814105
FY18 Approp: $385562872
FY18 Exp: $399924493

FY19 Adopted: $400425675
FY19 Approp: $400435574
FY19 Exp: $416762373

FY20 Adopted: $414237376
FY20 Approp: $414306876
FY20 Exp: $425553210

FY21 Adopted: $404182025
FY21 Approp: $404182025
FY21 Exp: $422917499

FY22 Adopted: $399871218
FY22 Approp: $400451303
 
FY23 Adopted: $395094796

Issue: We dont have a lot of data for proposed budget

Question: What do we learn by comparing proposed & adopted budgets? Wouldn't it be more useful to compare adopted budget & actual expenditures, or just measure all types of budgets + expenditure?

In [12]:
# BAR CHART
from plotly.graph_objs import *

year = ['FY15'] * 3 + ['FY16'] * 3 + ['FY17'] * 3 + ['FY18'] * 3 + ['FY19'] * 3 + ['FY20'] * 3 + ['FY21'] * 3 + ['FY22'] * 2 + ['FY23']
labels = ['Adopted Budget', 'Appropriated Budget', 'Expense'] * 7 + ['Adopted Budget', 'Appropriated Budget', 'Adopted Budget']
values = [319700081, 319700081, 337310895, 323509388, 332544971, 348887844, 356341193, 364321048, 364594820, 373814105, 385562872, 399924493, 400425675, 400435574, 416762373, 414237376, 414306876, 425553210, 404182025, 404182025, 422917499, 399871218, 400451303, 395094796]

layout = Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

fig5_dict = {
    'Year': year,
    'Key': labels,
    'Budget / Expense Value': values
}

df5 = pd.DataFrame(fig5_dict)
fig5 = px.bar(df5, x='Year', y='Budget / Expense Value', color='Key', barmode="group", color_discrete_sequence=['#BBBBBB','#231F20','#EF404E'])
fig5.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray')
fig5.update_traces(marker_line_width = 0)

# Font & other aesthetic stuff
fig5.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    font_family="GT America",
    font_color="black",
    xaxis_title=None,
    yaxis_title=None,
    yaxis_tickprefix = '$',
    height=500,
    width=1100,
    margin=dict(
        l=100,
        r=100,
        b=150,
        t=100,
        pad=4
    )
    
)

# Customize title!
fig5.update_layout(
    title=go.layout.Title(
        text="BPD budget has stabilized after FY2020, following an upward trend of budget increases",
        xref="paper",
        x=0,
        y=0.9,
        font_family="GT America Bold",
        font_size=19
    )
)

fig5.add_annotation(xref='paper',
            yref='paper',
            x=0, y=-0.4,
            showarrow=False,
            text ="Adopted Budget: The budget initially accepted by the city<br>Appropriated Budget: The budget that includes supplemental budget changes throughout the year<br>Expense: The actual expense calculated at the end of the fiscal year",
            font_size=13,
            font_color="gray",
            font_family="GT America",
            align="left"
            )

fig5.show()

Extension Project

Ideas for analyzing OT data:
- OT hours at Mass/Cass over the years & different mayors trying to improve the situation
- Analyze shift information --> Find trends


In [8]:
# Data Preprocessing
import datetime

# DATA
ot_data_df = pd.read_csv('./Data/Special_Events_Overtime_2012-2021/Special_Events_Overtime_2012-2022.csv')

# All dates must be in 21st century
def format_time(string):
    segments = string.split('/')
    month = segments[0]
    day = segments[1]
    year = segments[2]

    year = '20' + year
    date_str = month + '/' + day + '/' + year

    return datetime.datetime.strptime(date_str, '%m/%d/%Y')

# Only keep 'MASS / MELNEA' description rows
ot_data_df = ot_data_df.loc[ot_data_df['DESCRIPTION'] == 'MASS / MELNEA']


# Format dates
ot_data_df['OTDATE'] = ot_data_df['OTDATE'].apply(format_time)


ot_data_df.head()

,ID,NAME,RANK,ASSIGNED,ASSIGNED_DESC,CHARGED,CHARGED_DESC,OTDATE,OTCODE,DESCRIPTION,STARTTIME,ENDTIME,OTHOURS,DESCRIPTIONS,TYPE
144204,11212,"McCarthy,Michael P",LtDet,24111,C-11 DETECTIVES,40110,SPECIAL EVENTS,2019-08-01,449,MASS / MELNEA,1600,2000,4.00,SPECIAL EVENTS,4
144205,10167,"Doyle,Michael",Ptl,40010,COURT UNIT,40110,SPECIAL EVENTS,2019-08-01,449,MASS / MELNEA,1900,2345,4.75,SPECIAL EVENTS,4
144209,9035,"Power,Kevin T",SgtDet,24104,D-4 DETECTIVES,40110,SPECIAL EVENTS,2019-08-01,449,MASS / MELNEA,1900,2345,4.75,SPECIAL EVENTS,4
144212,116897,"Rooney,Sean M",Ptl,41010,DISTRICT 01,40110,SPECIAL EVENTS,2019-08-01,449,MASS / MELNEA,1900,2345,4.75,SPECIAL EVENTS,4
144215,144324,"Avila,Mariana Victoria",Ptl,42020,DISTRICT 02,40110,SPECIAL EVENTS,2019-08-01,449,MASS / MELNEA,1900,2345,4.75,SPECIAL EVENTS,4


In [9]:

date = []
ot_hours = []

grouped_df = ot_data_df[['OTDATE','OTHOURS']].groupby('OTDATE').sum()
print(grouped_df.head(10))

for index, row in grouped_df.iterrows():
    date.append(index.to_pydatetime())
    ot_hours.append(row['OTHOURS'])
    

dic = {
    'Date': date,
    'Overtime Hours': ot_hours,
}

df = pd.DataFrame(dic)
fig = px.line(df, x='Date', y="Overtime Hours", title='Police OT hours at Mass/Melnea')
fig.show()

# Rolling average graph BY 7 DAYS

date = []
ot_hours = []

rolling_avg = grouped_df['OTHOURS'].rolling(7).mean()
rolling_avg = rolling_avg.to_frame()

for index, row in rolling_avg.iterrows():
    date.append(index.to_pydatetime())
    ot_hours.append(row['OTHOURS'])

dic = {
    'Date': date,
    'Overtime Hours': ot_hours,
}
df = pd.DataFrame(dic)
fig2 = px.line(df, x='Date', y="Overtime Hours", title='Police OT hours at Mass/Melnea - 7 day rolling average')
fig2.show()


# Rolling average graph BY 14 DAYS

date = []
ot_hours = []

rolling_avg = grouped_df['OTHOURS'].rolling(14).mean()
rolling_avg = rolling_avg.to_frame()

for index, row in rolling_avg.iterrows():
    date.append(index.to_pydatetime())
    ot_hours.append(row['OTHOURS'])

dic = {
    'Date': date,
    'Overtime Hours per Day': ot_hours,
}
df = pd.DataFrame(dic)
fig3 = px.line(df, x='Date', y="Overtime Hours per Day", color_discrete_sequence=['#0055AA'])

# fig3.add_vline(x='Mar 22, 2021', line_width=2, line_dash="dash", line_color="red", annotation_text="Kim Janey becomes acting mayor of Boston")
date_in_ms = datetime.datetime.strptime("2021-11-16", "%Y-%m-%d").timestamp() * 1000
fig3.add_vline(x=date_in_ms, 
    line_width=2, 
    line_dash="dash", 
    line_color="red", 
    annotation_text="Michelle Wu becomes mayor of Boston ", 
    annotation_position="top left",
    annotation=dict(font_size=12, font_family="GT America")
)

# Font & other aesthetic stuff
fig3.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    font_family="GT America",
    font_color="black",
    xaxis_title=None,
    height=500,
    width=1200,
    margin=dict(
        l=100,
        r=100,
        b=100,
        t=100,
        pad=4
    )
)

# Customize title to add captions!
fig3.update_layout(
    title=go.layout.Title(
        text="Police overtime at Mass and Cass has grown over the years and has significantly increased during Mayor Wu's term<br><sup style='color:gray;'>The overtime hour values are represented by a 14 day rolling average</sup>",
        xref="paper",
        x=0,
        font_family="GT America Bold",
        font_size=18
    )
)

# Gridlines 
fig3.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray')

fig3.show()




            OTHOURS
OTDATE             
2019-08-01   119.50
2019-08-02   191.75
2019-08-03   110.00
2019-08-04   143.75
2019-08-05   128.75
2019-08-06   128.25
2019-08-07   128.25
2019-08-08   131.25
2019-08-09   114.00
2019-08-10   120.75


Extension Question 2: Hours when police are mot active at Mass/Melnea

Each bar in x-axis represents 1 hour of the day [12am, 1am ,2am, ... 10pm, 11pm]
y-axis is the hours spent

In [10]:
hours = {'12am': 0, 
    '1am': 0, 
    '2am': 0, 
    '3am': 0, 
    '4am': 0, 
    '5am': 0, 
    '6am': 0, 
    '7am': 0, 
    '8am': 0, 
    '9am': 0, 
    '10am': 0, 
    '11am': 0, 
    '12pm': 0, 
    '1pm': 0, 
    '2pm': 0, 
    '3pm': 0, 
    '4pm': 0, 
    '5pm': 0, 
    '6pm': 0, 
    '7pm': 0, 
    '8pm': 0, 
    '9pm': 0, 
    '10pm': 0, 
    '11pm': 0}

def record_hours(start, ot_hours):

    start_hr = -1
    start_min = -1

    # x100 to make it easier to work with 
    ot_hours = ot_hours * 100

    # SET start_hr AND start_min
    # 0-99 (12am - 12:59am)
    if (start >= 0 and start < 100):
        start_hr = 0
        start_min = start
    elif (start >= 100 and start < 1000):
        start_hr = int(str(start)[0])
        start_min = int(str(start)[1:])
    elif (start >= 1000 and start < 2400):
        start_hr = int(str(start)[:2])
        start_min = int(str(start)[2:])

    def getHoursToRecord(start_m):
        
        # Cases:
        # 1. Starting_Min = 0
        #   1-1. OT hours >= 100
        #   1-2. OT hours < 100
        # 2. Starting_Min > 0
        #   2-1. OT hours >= 100 - Starting_Min
        #   2-2. OT hours < 100 - Starting_Min

        # Time needed to reach next hour
        rem_time = 100 - start_m
        
        # Enough ot_hours to reach next hour
        if (ot_hours >= rem_time):
            return rem_time
        # Not enough ot_hours to reach next hour (IF CALLED, END HAS BEEN REACHED)
        elif (ot_hours < rem_time):
            return ot_hours

    def incrementHour(hr):
        hr = hr + 1
        if hr == 24:
            hr = 0
        return hr

    def getHourString(hr):
        string = ''

        if hr == 0:
            string = '12am'
        elif hr == 1:
            string = '1am'
        elif hr == 2:
            string = '2am'
        elif hr == 3:
            string = '3am'            
        elif hr == 4:
            string = '4am'
        elif hr == 5:
            string = '5am'
        elif hr == 6:
            string = '6am'
        elif hr == 7:
            string = '7am'
        elif hr == 8:
            string = '8am'
        elif hr == 9:
            string = '9am'            
        elif hr == 10:
            string = '10am'
        elif hr == 11:
            string = '11am'
        elif hr == 12:
            string = '12pm'
        elif hr == 13:
            string = '1pm'
        elif hr == 14:
            string = '2pm'
        elif hr == 15:
            string = '3pm'            
        elif hr == 16:
            string = '4pm'
        elif hr == 17:
            string = '5pm'
        elif hr == 18:
            string = '6pm'
        elif hr == 19:
            string = '7pm'
        elif hr == 20:
            string = '8pm'
        elif hr == 21:
            string = '9pm'            
        elif hr == 22:
            string = '10pm'
        elif hr == 23:
            string = '11pm'
        
        return string

    while (ot_hours != 0):

        # get hours to record
        hours_to_record = getHoursToRecord(start_min)

        # decrease ot hours by amount recorded
        ot_hours = ot_hours - hours_to_record

        # record hours in dictionary
        key = getHourString(start_hr)
        hours[key] = hours[key] + hours_to_record

        # increment start_hr 
        start_hr = incrementHour(start_hr)



df = ot_data_df[['OTDATE','OTHOURS']]
print(df.head(10))

for index, row in ot_data_df.iterrows():
    # date.append(index.to_pydatetime())
    record_hours(row['STARTTIME'], row['OTHOURS'])

print(hours.keys())
print(hours.values())

dic = {
    'Hours of day': hours.keys(),
    'Total Overtime Hours': hours.values(),
}

df = pd.DataFrame(dic)
fig = px.bar(df, x='Hours of day', y="Total Overtime Hours", color_discrete_sequence=['#0055AA'])

# Font & other aesthetic stuff
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    font_family="GT America",
    font_color="black",
    xaxis_title=None,
    height=500,
    width = 1200,
    margin=dict(
        l=100,
        r=100,
        b=100,
        t=100,
        pad=4
    ),
    bargap=0.1
)

# Customize title to add captions!
fig.update_layout(
    title=go.layout.Title(
        text="Police are most active at Mass and Cass from 4-7pm and somewhat active from 7am-4pm and 7pm-12am",
        xref="paper",
        x=0,
        font_family="GT America Bold",
        font_size=19
    )
)

# Gridlines 
fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray')

fig.show()

           OTDATE  OTHOURS
144204 2019-08-01     4.00
144205 2019-08-01     4.75
144209 2019-08-01     4.75
144212 2019-08-01     4.75
144215 2019-08-01     4.75
144216 2019-08-01     7.75
144217 2019-08-01     7.75
144218 2019-08-01     4.75
144219 2019-08-01     4.75
144220 2019-08-01     4.75
dict_keys(['12am', '1am', '2am', '3am', '4am', '5am', '6am', '7am', '8am', '9am', '10am', '11am', '12pm', '1pm', '2pm', '3pm', '4pm', '5pm', '6pm', '7pm', '8pm', '9pm', '10pm', '11pm'])
dict_values([257520.0, 249325.0, 248875.0, 248865, 249005.0, 249205, 250000.0, 877305.0, 877390.0, 870945.0, 882615.0, 882595.0, 885330.0, 662455.0, 633960.0, 630700.0, 1510065.0, 1490470.0, 1487960.0, 977825.0, 878975.0, 878045.0, 872865.0, 908680.0])
